# General information
This code is the central structure of `The Conference Calculator`. 

The sections of the basic version of the tool are:

1. Basic inputs (starting point, location of the conference).
2. Query conference database.
3. Route calculation for different modes of transportation (car, train, plane).
4. Carbon footprint calculation for different modes of transportation (car, train, plane).
5. Importance of the conference.
6. Outputs (2D plot).

## 0. Importations

In [65]:
import inquirer as iq
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import urllib.request
import os
import geopy.distance
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import osmnx as ox
from J0ANMM import carbon_car, carbon_flight, carbon_train
import wbgapi as wb
%matplotlib qt

## 1. Basic inputs
Questions asked (for now!):
1. What's your starting point?
2. Where is the conference?

In [97]:
list_of_cities = {
    'London':'St Pancras International',
    'Paris': 'Gare du Nord',
    'Brussels': 'Brussel-Centraal',
    'Amsterdam': 'Amsterdam Centraal',
    'Luxembourg': 'Luxembourg Railway Station',
    'Bern': 'Bern Railway Station',
    'Berlin':'Berlin Hauptbahnhof',
    'Copenhagen':'Copenhagen Central Station',
    'Prague': 'Prague Main Railway Station',
    'Ljubljana':'Ljubljana Train Station',
    'Vienna': 'Vienna International Busterminal (VIB)',
    'Oslo':'Oslo Central Station',
    'Bratislava': 'Bratislava Hlavna',
    'Madrid': 'Madrid Atocha',
    'Zagreb':'Zagreb Glavni Kolod Train Station',
    'Rome': 'Roma Termini',
    'Budapest': 'Budapest Keleti',
    'Warsaw': 'Warszawa Centralna railway station',
    'Stockholm':'Stockholm Central Station',
    'Sarajevo':'Sarajevo Main Railway station',
    'Lisbon':'Rossio Railway Station', 
    }

print("LIST OF AVAILABLE CITIES")
print("--------------------------------------")
for city in list_of_cities.keys(): print(city)
print("--------------------------------------")

# Departure city (starting point)
starting_point = input("Please enter your departure city: ")
if starting_point not in list_of_cities.keys(): 
    raise Exception("City not available! Enter another city.")

# Destination city (destination point)
destination_point = input("Please enter your destination city: ")
if destination_point not in list_of_cities.keys(): 
    raise Exception("City not available! Enter another city.")

# Create dictionary with both cities
journey_info = {'starting_point': starting_point, 'destination_point': destination_point}
journey_info

LIST OF AVAILABLE CITIES
--------------------------------------
London
Paris
Brussels
Amsterdam
Luxembourg
Bern
Berlin
Copenhagen
Prague
Ljubljana
Vienna
Oslo
Bratislava
Madrid
Zagreb
Rome
Budapest
Warsaw
Stockholm
Sarajevo
Lisbon
--------------------------------------


{'starting_point': 'London', 'destination_point': 'Vienna'}

## 2. Query conference database
TODO: Add Lorenzo code

## 3. Route calculation
Output: add distance from Google Maps API to `journey_info` for car, train and plane

In [98]:
# Read databases for train and car distances
car_distances_db = pd.read_csv(os.path.join(os.getcwd(), 'databases', 'Car_db.csv'), index_col=0)
train_distances_db = pd.read_csv(os.path.join(os.getcwd(), 'databases', 'Rail_db.csv'), index_col=0)

# Use database to retrieve distance between cities
journey_info['distance_car'] = car_distances_db[journey_info['starting_point']][journey_info['destination_point']] / 1000.
journey_info['distance_train'] = train_distances_db[journey_info['starting_point']][journey_info['destination_point']] / 1000.

# Retrieve latitude and longitude of both cities
latitude_starting, longitude_starting = ox.geocode_to_gdf(journey_info['starting_point']).lat.values[0], ox.geocode_to_gdf(journey_info['starting_point']).lon.values[0]
latitude_destination, longitude_destination = ox.geocode_to_gdf(journey_info['destination_point']).lat.values[0], ox.geocode_to_gdf(journey_info['destination_point']).lon.values[0]

# Use geopy to calculate the distance between the two cities (for plane)
journey_info['distance_plane'] = geopy.distance.geodesic((latitude_starting, longitude_starting), (latitude_destination, longitude_destination)).km

## 4. `gCO2` calculation based on transport mode and distance
Output: add carbon footprint from Google Maps API to `journey_info` for car, train and plane

In [102]:
# OSMNX information
country = ox.geocode_to_gdf(journey_info['destination_point'])['display_name'].values[0].split(', ')[-1]
wb_featureset = wb.economy.info(q=country)
country_ID = wb_featureset.items[0]['id'][:2]

if country_ID == 'GB': country_ID = 'UK'

# List of transport options
list_of_transport = ['petrol_car', 'electric_car', 'train', 'plane']

for mode in list_of_transport:

    # For electric and petrol cars
    if np.char.endswith(mode, 'car'):
        class_car = carbon_car.CarbonCar()
        co2_car = class_car.calculate_co2(dist_km=journey_info['distance_{}'.format(mode.split('_')[1])], \
                                       fuel_type=mode.split('_')[0], \
                                       pax_in_car=1, \
                                       trip_type='one-way')
        
        # Save carbon footprint in dictionary
        journey_info['carbon_footprint_{}'.format(mode)] = co2_car

    # For train
    elif mode == 'train':
        class_train = carbon_train.CarbonTrain()
        co2_train = class_train.calculate_co2(dist_km=journey_info['distance_{}'.format(mode)], \
                                              train_energy='electric', \
                                              train_country=country_ID, \
                                              trip_type='one-way')
        
        # Save carbon footprint in dictionary
        journey_info['carbon_footprint_{}'.format(mode)] = co2_train
    
    # For plane
    elif mode == 'plane':
        class_plane = carbon_flight.CarbonFlight()
        co2_plane = class_plane.calculate_co2(dist_km=journey_info['distance_{}'.format(mode)], \
                                              pax_class='economy', \
                                              trip_type='one-way')
        
        # Save carbon footprint in dictionary
        journey_info['carbon_footprint_{}'.format(mode)] = co2_plane


## 5. Importance of the conference
- *Are you presenting?* $\rightarrow$ talk, poster, invited talk, workshop lecture, etc.
- *How often is the conference held?* $\rightarrow$ e.g., every four years, unique conference, annually.
- *How specific is the conference to your research project?* $\rightarrow$ broad, specific, expert level?
- *Is there a relevant workshop/session you want/need to attend?*
- *Is the conference offered in hybrid mode?*
- *Are you an Early Career Researcher?*

In [53]:
class ConferenceImportance:
    def __init__(self):
        self.conference_importance = {'dictionaries': {},
                                      'information': {}}
        
        dict_ECR = {"Yes": 1.5,
                    "No": 1.}
        
        dict_presenting = { "I'm not presenting anything": 1,
                            'I was SELECTED to present a POSTER': 5,
                            'I was INVITED to present a POSTER': 6,
                            'I was SELECTED to give a TALK (less than 15 minutes including questions)': 7,
                            'I was INVITED to give a TALK (less than 15 minutes including questions)': 8,
                            'I was SELECTED to give a TALK (15 minutes or more including questions)': 8,
                            'I was INVITED to give a TALK (15 minutes or more including questions)': 9,
                            'I was INVITED to chair a session': 5,
                            'I was INVITED to be a panelist': 8,
                            'I was INVITED to give a workshop': 10,
                            'Other': None}
    
        dict_expertise = { 'Broad (e.g., all STEM conference, outreach conference)': 4,
                            'Field-specific (e.g., physics conference)': 5,
                            'Subfield-specific (e.g., astrophysics conference)': 6,
                            'Topic-specific (e.g., black hole conference)': 8,
                            'Research-specific (e.g., black hole feedback conference)': 10,
                            'Other': None}

        self.conference_importance['dictionaries']['dict_ECR'] = dict_ECR
        self.conference_importance['dictionaries']['dict_presenting'] = dict_presenting
        self.conference_importance['dictionaries']['dict_expertise'] = dict_expertise

    def asking_question_from_dict(self, dict_name):

        for idx, key in enumerate(self.conference_importance['dictionaries'][dict_name].keys()):
            print(idx+1, '-', key)

        user_info = int(input("Please select an option (1-{}):".format(len(self.conference_importance['dictionaries'][dict_name].keys()))))    
        user_info_score = self.conference_importance['dictionaries'][dict_name][list(self.conference_importance['dictionaries'][dict_name].keys())[user_info-1]]   

        return user_info_score

    def ask_ECR(self):
        print('\nQUESTION: Are you an Early Career Researcher (e.g., undergraduate student, postgraduate student, postdoctoral researcher, research assistant)?')
        user_info_ECR_score = self.asking_question_from_dict('dict_ECR')
        self.conference_importance['information']['info_ECR'] = user_info_ECR_score

    def ask_presenting(self):
        print('\nQUESTION: Are you presenting at the conference?')
        user_info_presenting_score = self.asking_question_from_dict('dict_presenting')

        if user_info_presenting_score == None:
            user_info_presenting_score = int(input('Please evaluate your contribution on a scale from 1 to 10: '))

        self.conference_importance['information']['info_presenting'] = user_info_presenting_score * self.conference_importance['information']['info_ECR']
    
    def ask_expertise(self):
        print('\nQUESTION: How specific is the conference to your field of expertise?')
        user_info_expertise_score = self.asking_question_from_dict('dict_expertise')

        if user_info_expertise_score == None:
            user_info_expertise_score = int(input('Please evaluate the importance (in terms of expertise) from 1 to 10: '))

        self.conference_importance['information']['info_expertise'] = user_info_expertise_score

    def main(self):
        self.ask_ECR()
        self.ask_presenting()    
        self.ask_expertise()

        return self.conference_importance

conference_importance = conferenceImportance().main()
        


QUESTION: Are you an Early Career Researcher (e.g., undergraduate student, postgraduate student, postdoctoral researcher, research assistant)?
1 - Yes
2 - No


ValueError: invalid literal for int() with base 10: ''

In [54]:
# Question: Are you an Early Career Researcher?


# Question: Are you presenting?
presentation_type = {"I'm not presenting": 0., 
                     "I'm presenting a poster": 5.,
                     "I'm presenting a talk": 8.,
                     "I'm giving a workshop": 15.,
                     "I was invited to give a talk": 12.}

presentation_type = input("Please enter your departure city: ")
if starting_point not in list_of_cities.keys(): 
    raise Exception("City not available! Enter another city.")

journey_info['importance'] = 6.

## 6. Outputs
### 6.1 UK average carbon footprint (used for plotting)

### 6.1 2D plot

In [56]:
fig, ax = plt.subplots()

def getImage(path):
   return OffsetImage(plt.imread(path, format="png"), zoom=0.15)

def mean(a, b):
    return (a+b)/2

paths = ['images\\plane.png', 'images\\electric_car.png', 'images\\train.png']

#with plt.xkcd():
average_UK = 12700 # tonnes CO2e
average_UK = 4000 # tonnes CO2e

percentages = np.array([0., 0.5, 1., 1.5, 2., 5.])
cf_percentages = average_UK * percentages

list_c = ['red', 'green', 'blue', 'orange', 'purple']

for idx in range(len(cf_percentages)-1):
    print(cf_percentages[idx], cf_percentages[idx+1])
    ax.axvspan(cf_percentages[idx], cf_percentages[idx+1], alpha=0.2, color=list_c[idx])
    ax.text(mean(cf_percentages[idx], cf_percentages[idx+1]), 8, str(percentages[idx])+'–'+str(percentages[idx+1])+"x", color=list_c[idx], ha='center', fontsize=15)

for idx, transport in enumerate(['plane', 'car', 'train']):
   print(idx, transport)
   print(journey_info['carbon_footprint_{}'.format(transport)], journey_info['importance'])
   ab = AnnotationBbox(getImage(paths[idx]), (journey_info['carbon_footprint_{}'.format(transport)], journey_info['importance']), frameon=False)
   ax.add_artist(ab)

plt.xlim(0, 6000)
plt.ylim(0, 10)
plt.xlabel("Carbon footprint (tonnes CO2e)", fontsize=15)
plt.ylabel("Importance of the conference", fontsize=15)
plt.title('Conference in {} (departure from {})'.format(journey_info['destination_point'], journey_info['starting_point']), fontsize=15)
plt.show()

0.0 2000.0
2000.0 4000.0
4000.0 6000.0
6000.0 8000.0
8000.0 20000.0
0 plane
5000.0 6.0
1 car
1500.0 6.0
2 train
900.0 6.0
